<a href="https://colab.research.google.com/github/SartajHundal/DroneSimulations/blob/main/DroneLocalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from PIL import Image
from google.colab import drive
import json
import math
import numpy as np
import pandas as pd
import random

# NEW (22.02.24): When you are doing a mv in Google Drive, the Colab
# notebook does not sync on the left-hand 'Files' pane until refreshing

In [10]:
def load_occupancy_map(map_path):
    """
    Load the occupancy map from a PNG file.

    Parameters:
    - map_path (str): Path to the PNG file containing the occupancy map.

    Returns:
    - occupancy_map (numpy.ndarray): Occupancy map of the environment.
    """
    # Load the PNG image
    map_image = Image.open(map_path)

    # Convert the image to grayscale
    map_gray = map_image.convert('L')

    # Convert the image to a numpy array
    occupancy_map = np.array(map_gray)

    return occupancy_map


def load_sensor_log(log_path):
    with open(log_path, 'r') as f:
        return [json.loads(line) for line in f]

In [11]:
def measure_distance(occupancy_map, x0, y0, max_steps=200):
    """
    Simulate lidar measurements.

    Parameters:
    - occupancy_map (numpy.ndarray): Occupancy map of the environment.
    - x0 (float): x-coordinate of the drone's current position.
    - y0 (float): y-coordinate of the drone's current position.
    - max_steps (int): Maximum number of steps for lidar measurements.

    Returns:
    - measurements (list): List of lidar measurements.
    """
    measurements = []
    map_height, map_width = occupancy_map.shape

    for angle_degrees in range(0, 360, 10):
        angle_radians = math.radians(angle_degrees)
        vx = math.cos(angle_radians)
        vy = math.sin(angle_radians)
        x = x0
        y = y0

        for step in range(max_steps):
            x += vx
            y += vy

            if x >= map_width or x < 0 or y >= map_height or y < 0:
                measurements.append(-1)
                break

            if occupancy_map[int(y), int(x)] > 0:
                distance = math.sqrt((x - x0)**2 + (y - y0)**2)
                measurements.append(distance)
                break

            if step == max_steps - 1:
                measurements.append(-1)

    return measurements

In [5]:
def add_odometry_noise(odom_x, odom_y, variance=4.4):
    """
    Add Gaussian noise to odometry measurements.

    Parameters:
    - odom_x (float): Odometry measurement for x-axis.
    - odom_y (float): Odometry measurement for y-axis.
    - variance (float): Variance of the Gaussian noise.

    Returns:
    - noisy_odom_x (float): Noisy odometry measurement for x-axis.
    - noisy_odom_y (float): Noisy odometry measurement for y-axis.
    """
    noisy_odom_x = odom_x + random.gauss(0, math.sqrt(variance))
    noisy_odom_y = odom_y + random.gauss(0, math.sqrt(variance))
    return noisy_odom_x, noisy_odom_y

In [6]:
def add_lidar_noise(measurements, variance=0.1):
    """
    Add Gaussian noise to lidar measurements.

    Parameters:
    - measurements (list): List of lidar measurements.
    - variance (float): Variance of the Gaussian noise.

    Returns:
    - noisy_measurements (list): List of noisy lidar measurements.
    """
    return [distance + random.gauss(0, math.sqrt(variance)) if distance != -1 else -1 for distance in measurements]

In [7]:
def determine_drone_path(occupancy_map, sensor_log):
    """
    Determine the drone's path based on sensor data and occupancy map.

    Parameters:
    - occupancy_map (numpy.ndarray): Occupancy map of the environment.
    - sensor_log (list): List of sensor log data.

    Returns:
    - drone_path (list): List of coordinates representing the drone's path.
    """
    x, y = 0, 0
    drone_path = [(x, y)]

    for frame_data in sensor_log:
        noisy_odom_x, noisy_odom_y = add_odometry_noise(frame_data['odometry_x'], frame_data['odometry_y'])
        x += noisy_odom_x
        y += noisy_odom_y

        lidar_measurements = measure_distance(occupancy_map, x, y)
        noisy_lidar_measurements = add_lidar_noise(lidar_measurements)

        min_distance = float('inf')
        next_x, next_y = x, y
        for angle, distance in enumerate(noisy_lidar_measurements):
            if distance != -1 and distance < min_distance:
                min_distance = distance
                next_x = x + distance * math.cos(math.radians(angle * 10))
                next_y = y + distance * math.sin(math.radians(angle * 10))

        x, y = next_x, next_y
        drone_path.append((x, y))

    return drone_path

In [26]:
# Load occupancy map and sensor log data (update paths as needed)
occupancy_map_path = "/content/drive/MyDrive/Drone (shared)/map_stadi_bg.png"
sensor_log_path = "/content/drive/MyDrive/Drone (shared)/drone_log.jsonl"

occupancy_map = load_occupancy_map(occupancy_map_path)
sensor_log = load_sensor_log(sensor_log_path)

# Determine drone's path
drone_path = determine_drone_path(occupancy_map, sensor_log)
df = pd.DataFrame(drone_path, columns=['x', 'y'])
print(df)

# Read JSONL file into a DataFrame
df_0 = pd.read_json(sensor_log_path, lines=True)

# Display the DataFrame for EDA; we'll use
# this for Pearson, MAPE / MASE, etc. later
print(df_0)


              x           y
0      0.000000    0.000000
1     93.660322    7.592221
2     80.983360   -7.544099
3     69.591479  -20.176928
4     60.819219  -28.007333
..          ...         ...
132  400.650049 -615.626539
133  392.045948 -638.213420
134  381.777689 -665.136259
135  368.499658 -690.677819
136  371.015803 -725.095531

[137 rows x 2 columns]
     odometry_x  odometry_y                                              lidar
0     -0.380680    5.874107  [31.046601487401276, 31.657418178520697, 33.50...
1    -12.140913  -10.468727  [30.868371257740733, 32.55997471557303, 34.364...
2    -12.171528  -10.099245  [32.52217980945568, 31.302490262596827, 32.898...
3     -6.167448   -4.434787  [30.398328674199924, 32.22570009019953, 45.403...
4      2.103860  -19.712639  [31.112962867196075, 30.934269232710083, 34.64...
..          ...         ...                                                ...
131   -1.867480   -9.496820  [77.72277427567646, 183.16464966991614, 156.74...
132   -7